In [1]:
import numpy as np
import pandas as pd

In [2]:
from scipy.spatial.distance import squareform, pdist

In [3]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [4]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [5]:
import sys

In [6]:
from sklearn.metrics.pairwise import pairwise_distances

In [7]:
from sklearn.metrics import f1_score

In [8]:
def labelClusters(clusters, target):
    mySet = set(clusters)
    resultList = []
    for i in mySet:
        myClasses = []
        for j in range(len(clusters)):
            if (clusters[j] == i):
                myClasses.append(target[j]) 
        resultList.append(max(set(myClasses), key=myClasses.count))
    
    mySetArray = []
    for i in mySet:
        mySetArray.append(i)
        
    labels = []
        
    for i in range(len(clusters)):
        for j in range(len(mySetArray)):
            if (clusters[i] == mySetArray[j]):
                labels.append(resultList[j])
    return labels

In [9]:
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

In [91]:
len(set([1,2,1,3,4]))

4

In [10]:
def counting(arr):
    mySet = set(arr)
    myList = []
    myNewSet = []
    for i in mySet:
        myNewSet.append(i)
        
    for i in range(len(myNewSet)):
        myList.append(0)
        for j in arr:
            if (myNewSet[i]==j):
                myList[i] += 1
    return myList

In [11]:
def menorCluster(clusters):
    mySet = set(clusters)
    minimo = len(clusters)
    menor = len(clusters)
    for i in mySet:
        if((clusters.count(i)) < minimo):
            minimo = clusters.count(i)
            menor = i
            
    myList = []
    
    for j in range(len(clusters)):
        if(clusters[j]==menor):
            myList.append(j)
            
    return myList

In [144]:
def genieLinkage(data, limiar, clusterNumber):
    
    clusters = {}
    row_index = 0
    col_index = 0
    array = []
    

    for n in range(data.shape[0]):
        array.append(n)
    
        
    clusters[0] = array.copy()

    #finding minimum value from the distance matrix
    #note that this loop will always return minimum value from bottom triangle of matrix
    for k in range(1, data.shape[0]):
        min_val = sys.maxsize
        
        if(gini(counting(clusters[k-1])) <= limiar):
            for i in range(0, data.shape[0]):
                for j in range(i, data.shape[1]):
                    if(data[i][j]<min_val):
                        min_val = data[i][j]
                        row_index = i
                        col_index = j
        else:
            iList = menorCluster(clusters[k-1])
            for i in iList:
                for j in range(0, data.shape[0]):
                    if(j not in iList):
                        if(data[i][j]<min_val):
                            min_val = data[i][j]
                            row_index = i
                            col_index = j

        array = clusters[k-1].copy()
        minimum = min(array[row_index],array[col_index])
        maximum = max(array[row_index],array[col_index])
        
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        for i in range(len(array)):
            for j in range(len(array)):
                if(array[i]==array[j]):
                    data[i][j] = sys.maxsize
                    
        if (len(set(clusters[k])) == clusterNumber):
            print(set(clusters[k]))
            return clusters[k]

In [93]:
def genie(dataset, target, limiar, clusterNumber):
    
#     dataX = dataset.loc[:,dataset.columns[:-1]].values
#     dataY = dataset.loc[:,[dataset.columns[-1]]].values
    dataX = dataset.values
    dataY = target[0]
    
    dataDistances = pairwise_distances(dataX,metric='euclidean')
    #making all the diagonal elements infinity 
    np.fill_diagonal(dataDistances,sys.maxsize)
    
    clusters = genieLinkage(dataDistances, limiar, clusterNumber)
    
    labels = labelClusters(clusters, dataY)
    
    fmScore = f1_score(dataY, labels, average='micro')
    
    return fmScore

In [14]:
from sklearn.datasets import load_iris
iris = load_iris()

In [15]:
genie(iris, 0.4)

0.97333333333333338

In [17]:
genie(iris, 1)

0.97999999999999998

In [116]:
a1 = pd.read_table("datasets/a1.data", header=None, delim_whitespace=True)
iris5 = pd.read_table("datasets/iris5.data", header=None, delim_whitespace=True)
flame = pd.read_table("datasets/flame.data", header=None, delim_whitespace=True)
pathbased = pd.read_table("datasets/pathbased.data", header=None, delim_whitespace=True)
unbalance = pd.read_table("datasets/unbalance.data", header=None, delim_whitespace=True)

In [117]:
a1labels = pd.read_table("labels/a1.labels", header=None, delim_whitespace=True)
iris5labels = pd.read_table("labels/iris5.labels", header=None, delim_whitespace=True)
flamelabels = pd.read_table("labels/flame.labels", header=None, delim_whitespace=True)
pathbasedlabels = pd.read_table("labels/pathbased.labels", header=None, delim_whitespace=True)
unbalancelabels = pd.read_table("labels/unbalance.labels", header=None, delim_whitespace=True)

In [94]:
genie(iris5, iris5labels, 0.3, 3)

0.89523809523809528

In [95]:
genie(iris5, iris5labels, 0.2, 3)

0.89523809523809528

In [99]:
genie(iris5, iris5labels, 0.4, 3)

0.89523809523809528

In [97]:
genie(iris5, iris5labels, 0.5, 3)

0.5714285714285714

In [98]:
genie(iris5, iris5labels, 0.6, 3)

0.5714285714285714

In [140]:
genie(iris5, iris5labels, 1, 3)

104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3


0.54285714285714282

In [147]:
genie(pathbased, pathbasedlabels, 0.3, 3)

{0, 202, 56}


0.82333333333333336

In [118]:
len(set(unbalancelabels[0]))

8